# An open investigation for the theft of a store using image preprocessing.
### This is a CCTV Mug Shot Footage - Robbery 16th February 2016 for robbers who took SONY cameras from a shop in London.


In [ ]:
import os
import pandas as pd 
import numpy as np
import cv2
import urllib.request
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns 
%matplotlib inline
from skimage import exposure
from PIL import Image, ImageEnhance , ImageFilter 
import random
from imgaug import augmenters, imgaug



# hide warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
im = Image.open('../input/hhhhhhhhh/Screenshot (10).png')
implot = plt.imshow(im)
plt.show()

In [ ]:
xcenter = im.width/2
ycenter = im.height/2
x1 = xcenter - 20
y1 = ycenter - 400
x2 = xcenter + 120
y2 = ycenter - 240
cropped = im.crop((x1, y1, x2, y2))
cropped.save('cropped.png')
cropped.show
implot = plt.imshow(cropped)
plt.show()

img = cropped.copy()


# Let's try to clear his face
### We add colors to the image

In [ ]:
enhancer = ImageEnhance.Color(img)
img = enhancer.enhance(1.3)
implot = plt.imshow(img)
plt.show()


### We add more brightness

In [ ]:
enhancer = ImageEnhance.Brightness(img)
img = enhancer.enhance(1.2)

## We make the image more sharpened 

In [ ]:
enhancer = ImageEnhance.Sharpness(img)
img = enhancer.enhance(5)
implot = plt.imshow(img)
plt.show()


### We add more contrast

In [ ]:
enhancer = ImageEnhance.Contrast(img)
img = enhancer.enhance(1.2)
img.save('enhanced.png')
implot = plt.imshow(img)
plt.show()



## Now let's take a look at a histogram of the color channels in the face picture:


In [ ]:
from skimage import io
import matplotlib.pyplot as plt

def plot_color_hist(img):
    plt.figure(figsize=(10,5))
    plt.subplot(1,3,1)
    plt.title('Red')
    plt.hist(img[:,:,0].ravel(), bins=256, color='r')
    plt.subplot(1,3,2)
    plt.title('Green')
    plt.hist(img[:,:,1].ravel(), bins=256, color='g')
    plt.subplot(1,3,3)
    plt.title('Blue')
    plt.hist(img[:,:,2].ravel(), bins=256, color='b')
    plt.show()

img = io.imread('enhanced.png')
plot_color_hist(img)


## Now let's play detectives here and try to find the face's edges in the image using canny edge detection method:


In [ ]:
edges = cv2.Canny(np.array(img), 100, 200)
implot = plt.imshow(edges)
plt.show()


## But hear me out, this surviliance cam is actually a good one, now let's make a worse scenario :(
### add blurring to the original image


In [ ]:
implot = plt.imshow(im)
plt.show()
im_blur = im.filter(ImageFilter.GaussianBlur(radius=4))
im_blur.save('blur.png')
implot = plt.imshow(im_blur)
plt.show()

### Now let's try unblurring it

In [ ]:
# deblur the image
im = Image.open('blur.png')
implot = plt.imshow(im)
plt.show()
im_deblur = im.filter(ImageFilter.UnsharpMask(radius=5, percent=500, threshold=3))
im_deblur.save('deblur.png')
implot = plt.imshow(im_deblur)
plt.show()


## Maybe a worse scenario is when the camera is balck andwhite not working properly:

In [ ]:
implot = plt.imshow(im)
plt.show()
im_noise = augmenters.AdditiveGaussianNoise(scale=60)(images=np.array(im))
implot = plt.imshow(im_noise)
plt.show()



### adding blur to the original image by additive gaussian noise


In [ ]:
implot = plt.imshow(im)
plt.show()
im_blur = augmenters.AdditiveGaussianNoise(scale=60)(images=np.array(im))
im_blur = Image.fromarray(im_blur)
im_blur = im_blur.filter(ImageFilter.GaussianBlur(radius=4))
implot = plt.imshow(im_blur)
plt.show()



### Adding salt and pepper noise

In [ ]:
import imgaug.augmenters as iaa
aug = iaa.SaltAndPepper(0.25, per_channel=True)
implot = plt.imshow(im)
plt.show()
im_salt = aug(images=np.array(im))
implot = plt.imshow(im_salt)
plt.show()


### Convert to grayscale


In [ ]:
implot = plt.imshow(im)
plt.show()
im_gray = im.convert('LA')
implot = plt.imshow(im_gray)
im_gray.save('gray.png')
plt.show()


## Binarize the image using thresholding 


In [ ]:
col = Image.open("gray.png")
gray = col.convert('L')
bw = gray.point(lambda x: 0 if x<85 else 255, '1')
bw.save("result_bw.png")
im = Image.open('result_bw.png')
implot = plt.imshow(im)
plt.show()